### PCP Assignment 2 Module 2 - similarity_module

In [1]:
from load_dataset_module import Artist, Song, Track, IterRegistry, Extras, File_loader   # Classes
import pandas as pd
import numpy as np
import math
from scipy.spatial import distance 
from numpy.linalg import norm
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import cosine_similarity

# Disable warnings, these are not required for the user to see (might be removed)
import warnings
warnings.filterwarnings('ignore')

# These will change over time
#from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
#from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
#from sklearn.neural_network import MLPClassifier

In [2]:
# Creating logging capabilities

# import logging
# logging.warning('Watch out!')  # will print a message to the console
# logging.info('I told you so')  # will not print anything

In [3]:
song_searcher = File_loader().read_file()

Finished reading the command for file loading.


In [4]:
music_df = pd.DataFrame.from_records([s.to_dict() for s in song_searcher])

full_df = music_df

music_df = music_df.drop(['Artist/s', 'Song Name'], axis = 1)

features_df = music_df

scaler = MinMaxScaler()
music_df = scaler.fit_transform(music_df)

print(features_df[:2])
print('\n', music_df[:2])

   Acousticness  Danceability  Energy  Liveness  Loudness  Popularity  \
0         0.995         0.708   0.195     0.151   -12.428           0   
1         0.604         0.749   0.220     0.119   -19.924           0   

   Speechiness    Tempo  Valence  Explicit  Instrumentalness  
0       0.0506  118.469    0.779         0             0.563  
1       0.9290  107.177    0.880         0             0.000  

 [[0.99899598 0.71659919 0.195      0.151      0.74500039 0.
  0.05221878 0.48534768 0.779      0.         0.563     ]
 [0.6064257  0.75809717 0.22       0.119      0.62760943 0.
  0.95872033 0.43908624 0.88       0.         0.        ]]


In [5]:
full_df.head()

,Artist/s,Song Name,Acousticness,Danceability,Energy,Liveness,Loudness,Popularity,Speechiness,Tempo,Valence,Explicit,Instrumentalness
0,['Carl Woitschach'],Singende Bataillone 1. Teil,0.995,0.708,0.195,0.151,-12.428,0,0.0506,118.469,0.779,0,0.563
1,['Seweryn Goszczyński'],Chapter 1.18 - Zamek kaniowski,0.604,0.749,0.220,0.119,-19.924,0,0.9290,107.177,0.880,0,0.000
2,['Francisco Canaro'],Bebamos Juntos - Instrumental (Remasterizado),0.995,0.781,0.130,0.111,-14.734,0,0.0926,108.003,0.720,0,0.887
3,['Carl Woitschach'],Per aspera ad astra,0.988,0.555,0.421,0.105,-9.878,0,0.0474,123.310,0.857,0,0.836
4,"['Francisco Canaro', 'Charlo']",Moneda Corriente - Remasterizado,0.995,0.683,0.207,0.337,-9.801,0,0.1270,119.833,0.493,0,0.206


In [6]:
song_searcher[15665]

Artist/s: 'Edna St. Vincent Millay', Song Name: Fatal Interview: Sonnet XI, ID: 2ALjYTnVOy0LINGcpm0LqA, Acousticness: 0.992, Danceability: 0.6809999999999999, Energy: 0.324, Liveness: 0.11900000000000001, Loudness: -15.855, Popularity: 3, Speechiness: 0.935, Tempo: 127.41, Valence: 0.265

In [7]:
song_searcher[15665].getExtras()

'Duration: 83952, Explicit: 0, Instrumentalness: 0.436, Key: 8, Mode: 0, Release Date: 1955'

In [12]:
class Searcher(object):
    __metaclass__ = IterRegistry
    _registry = []
    
    def __init__(self, list_name):
        self.list_name = list_name
        
    def search_artist(self):
        try:
            fName = str(input("Please enter the first name of the artist you want to find: ").strip().capitalize())
            lName = str(input("Please enter the surname of the artist you want to find: "\
                             ).strip().capitalize())
    
            # Create empty lists for the inputs
            result_names = []
            result_songs = []
            result_id = []
            num = 0 # Index for printing results
            
            # Loop through the dictionary, extract all matching artist + song values
            for i in range(len(self.list_name)): # range of the whole class-based list
                if lName != "":  # If the last name entry is not left blank
                    if fName + " " + lName in self.list_name[i].getName():
                        result_names.append(self.list_name[i].getName())
                        result_songs.append(self.list_name[i].getSongName())
                        result_id.append(i) 
                else:  # If the artist only has one name e.g. Nirvana
                    if fName in self.list_name[i].getName():
                        result_names.append(self.list_name[i].getName())
                        result_songs.append(self.list_name[i].getSongName())
                        result_id.append(i) 
                
            if len(result_id) == 0: # If the length of the list above is 0, we found no matches
                print("Your search returned no results.")
            else:
                # return an assignable dictionary using the First Name and Surname Initial for the indexable ID
                print("Your search returned {} results.".format(len(result_id)))
                output = str(input("Would you like to view results? ").strip().capitalize())
                if output == "Yes":
                    for k in range(0, len(result_id)):
                        print("ID: {} | {} | {} | ".format(result_id[num], result_names[num], result_songs[num]))
                        num+=1
                else:
                    print("Search complete.")
            
            return result_id  # Return the id to pull details if wanted
    
        # Error handling for the function is written here
        except KeyError:
            return(print("You have entered an incorrect value, please check your entry."))
        except TypeError:
            return(print("You can't enter a number or symbol here, please enter a string dictionary name."))
        except IndexError:
            return(print("You must enter at least an Initial into the Surname box."))
        
    def search_song(self):
        try:
            song_name = input("Please insert the word/s you would like to find in a song: ").capitalize()
            song_name = song_name.rstrip()   # remove end of input whitespace
            song_name = song_name.split(' ') # split the words into individual words to detect how many we have

            result_id = []
            result_names = []
            result_songs = []
            num = 0 # Index for printing results
        
            if len(song_name) == 1: # Check if the input is 1 word or many words
                song_name = ''.join(song_name) # if it is 1 word, reconnect the word through the join command
                for i in range(0, len(self.list_name)): # range of the whole class-based list
                    if song_name in self.list_name[i].getSongName():
                        # Add matching values to assignable lists 
                        result_id.append(i)      
                        result_names.append(self.list_name[i].getName())
                        result_songs.append(self.list_name[i].getSongName()) 
                    
            else: # Loop the list and match each word with a value
                for i in range(0, len(self.list_name)): # range of the whole dictionary +1 as end of dictionary is missed otherwise
                    song_name = [item.capitalize() for item in song_name] # capitalise each string
                    for j in range(0, len(song_name)):
                        if song_name[j] in self.list_name[i].getSongName(): # search each word for a match
                            # Add matching values to assignable lists
                            result_id.append(i)     
                            result_names.append(self.list_name[i].getName())
                            result_songs.append(self.list_name[i].getSongName())

            if len(result_id) == 0: # If the length of the list is 0, we found no matches
                print("Your search returned no results.")
            else:
                print("Your search returned {} results.".format(len(result_id)))
                output = str(input("Would you like to view results? ").strip().capitalize())
                if output == "Yes":
                    for k in range(0, len(result_id)):
                        print("ID: {} | {} | {}".format(result_id[num], result_names[num], result_songs[num]))
                        num+=1
                else:
                    print("Search complete.")
                    
        # Error handling for the function is written here
        except KeyError as keyerror:
            return(print("You have entered an incorrect value, please check your entry.", keyerror))
        except TypeError as typeerror:
            return(print("You can't enter a number or symbol here, please enter a string dictionary name.", typeerror)) 

In [9]:
class Similarity_metric(object):
    __metaclass__ = IterRegistry
    _registry = []
    
    def __init__(self, list_name, value1, value2):
        self.list_name = list_name
    # These do work for x by x conversion however
        # Our defined target chosen by the user
        self.target = value1
        # Our defined library of all other values
        self.library = value2
        
    # Returns object in string format for readability
    def __repr__(self):
        return f'Target: {self.target}, Comparison: {self.library}'
    
    def euclidean(self):
        a = self.target
        b = self.library
        return np.linalg.norm(a - b)
        
    def manhattan(self):
        a = self.target
        b = self.library
        #return np.abs(a - b) # This works
        return manhattan_distances([a, b])[1,0]
        
    def cosine(self):
        a = self.target
        b = self.library
        return 1 - distance.cosine(a, b)

    def jaccard(self):
        a = self.target
        b = self.library
        return distance.jaccard(a, b)
    
    def pearson(self):
        a = self.target
        b = self.library
        return np.corrcoef([a, b])[1,0]
    
    def feature_select(self):
        feature_select = ["Accoustiness", "Danceability", "Energy", "Liveness", "Loudness", "Popularity", "Speechiness", "Tempo", "Valence", "Explicit", "Instrumentalness"]
        for number, feature in enumerate(feature_select, start=1):
            print(number, feature) # Present a list of options to the user for feature choice
    
    def metric_choice(self):
        id1 = self.target
        id2 = self.library
        
        metric = int(input("Which metric would you like to use from the selection: Enter the number: "))
        if metric == 1:
            return Similarity_metric(self.list_name, id1, id2).euclidean()
        elif metric == 2:
            return Similarity_metric(self.list_name, id1, id2).cosine()
        elif metric == 3:
            return np.corrcoef([id1, id2]) # Issues with Pearson, do it manually
        elif metric == 4:
            return Similarity_metric(self.list_name, id1, id2).jaccard()
        elif metric == 5:
            return Similarity_metric(self.list_name, [id1], [id2]).manhattan()
        else:
            print("Your selection is incorrect. Defaulting to Euclidean.")
            return Similarity_metric(self.list_name, id1, id2).euclidean()
    
    def metric_selection(self):
        metric_select = ["Euclidean", "Cosine", "Pearson", "Jaccard", "Manhattan"]
        for number, metric in enumerate(metric_select, start=1):
            print(number, metric) # Present a list of options to the user for metric choice 
    
    # Assignment 1 section as OOP
    def measure_feature(self):
        list_name = self.list_name
        id1 = self.target
        id2 = self.library
        
        try:
            id1 = id1
            id2 = id2
            #id1 = int(input("Please insert your first id for music features: "))
            #id2 = int(input("Please insert your second id for music features: "))

            if id1 == id2: # check to see if the IDs match
                print("Similarity measure for ID {} and ID {} is 1".format(id1, id2))
            else:
                Similarity_metric(list_name, id1, id2).feature_select()
                query = input("Which feature do you want to use for comparison? Enter the feature name or enter 'No' to compare all features. ").strip().capitalize()

                if query == '' or query == "No".capitalize(): # if query entry is no or left empty, go here
                    print("Comparing all respective features.\n")
                    Similarity_metric(list_name, id1, id2).metric_selection()
                    response = int(input("Which metric would you like to use from the selection: Enter the number: "))
                    # take values from the dictionary from the end,to avoid the string values at the beginning
                    feature_list = list(list_name.iloc[id1])
                    feature_list2 = list(list_name.iloc[id2])
                    column_list = [str(i) for i in list_name] # take the associated column names
            
                    for i in range(0,11): # Loop through the 11 expected features
                        # take values over the loop and compare them
                        for value in feature_list, feature_list2: 
                            x = (feature_list[i])
                            y = (feature_list2[i])
                            if response == 1:
                                distance = Similarity_metric(list_name, x, y).euclidean()
                            elif response == 2:
                                distance = Similarity_metric(list_name, x, y).cosine()
                            elif response == 3: # Issues with Pearson, do it manually
                                distance = np.corrcoef([x, y])
                            elif response == 4:
                                distance = Similarity_metric(list_name, x, y).jaccard()
                            elif response == 5:
                                distance = Similarity_metric(list_name, [x],[y]).manhattan()
                            else:
                                response = 1
                                print("Your selection is incorrect. Defaulting to Euclidean.")
                                distance = Similarity_metric(list_name, x, y).euclidean()
                        print(column_list[i] + ':', round(distance, 3)) # print all feature metrics
                else:
                    x = list_name[query][id1]
                    y = list_name[query][id2]
                    Similarity_metric(list_name, id1, id2).metric_selection()
                    distance = Similarity_metric(list_name, x, y).metric_choice() 
                    print("Similarity of {} for ID {} and ID {} is".format(query, id1, id2), round(distance, 3))
       
        except KeyError as keyerror:
            print("That feature doesn't exist.", keyerror)
            #euclidean_similarity(list_name,'','')
        except ValueError as valueerror:
            print("Your entry is invalid, please make sure your entry was the correct format.", valueerror)
            #euclidean_similarity(list_name,'','')
        except TypeError as typeerror:
            print("Invalid type entered.", typeerror)
        #except IndexError as index:
        #    print("There was a problem, did you enter your dictionary name correctly?", index)
        except ZeroDivisionError:
            print("Sorry, but you cannot divide by 0, metric will restart.")
            #euclidean_similarity(list_name,'','')
        except AttributeError as attrerror:
            print(attrerror)
            #euclidean_similarity(list_name,'','')

In [10]:
song = Similarity_metric(features_df, 340, 200).measure_feature()
song

1 Accoustiness
2 Danceability
3 Energy
4 Liveness
5 Loudness
6 Popularity
7 Speechiness
8 Tempo
9 Valence
10 Explicit
11 Instrumentalness
Which feature do you want to use for comparison? Enter the feature name or enter 'No' to compare all features. tempo
1 Euclidean
2 Cosine
3 Pearson
4 Jaccard
5 Manhattan
Which metric would you like to use from the selection: Enter the number: 1
Similarity of Tempo for ID 340 and ID 200 is 48.63


In [13]:
search = Searcher(song_searcher).search_artist()

Please enter the first name of the artist you want to find: nirvana
Please enter the surname of the artist you want to find: 
Your search returned 83 results.
Would you like to view results? no
Search complete.


In [14]:
print(search)

# #full_df.iloc[31]['Artist/s'].strip("'[]'").split(" ")

# features_to_take = []
# #print(features_to_take.empty)
# #features_to_take = scaler.fit_transform(features_to_take)

# for i in range(len(search)):
#     print(features_df['Acousticness'][i])
    
# #print(features_to_take[:1])

[4639, 4657, 19108, 20456, 20467, 27679, 27701, 33282, 40472, 41549, 47468, 47627, 54684, 61545, 68457, 69270, 76465, 76482, 76486, 84379, 84444, 84453, 84677, 84685, 84706, 84731, 84876, 85588, 92975, 93153, 93421, 101702, 102030, 102115, 102158, 110737, 110852, 110951, 111028, 111067, 112644, 119661, 119838, 119872, 119873, 119935, 120048, 121173, 128181, 128241, 128398, 128415, 128454, 128590, 128607, 128627, 136725, 136942, 137025, 137124, 137140, 138021, 145074, 145087, 145306, 145311, 145345, 145370, 145411, 145440, 145470, 145506, 145508, 145545, 145557, 153638, 153661, 153703, 153776, 153798, 153802, 153864, 154978]


In [15]:
#### Class for recommendations
class Recommendation(Similarity_metric):
    
    def __init__(self, list_name):
        self.list_name = list_name
        
    def metric_choice(self):
        metric = int(input("Which metric would you like to use from the selection: Enter the number: "))
        if metric == 1:
            return 1
        elif metric == 2:
            return 2
        elif metric == 3:
            return 3
        elif metric == 4:
            return 4
        elif metric == 5:
            return 5
        else:
            print("Your selection is incorrect. Defaulting to Euclidean.")
            return 1   

    def get_artist_recommendation(self):
        list_name = self.list_name
        
        # Define needed variables
        results = []
        id_num = int(input("Please enter the ID number of an artist: "))
        i = 0
        j = 0
        
        Recommendation(list_name).metric_selection()
        response = Recommendation(list_name).metric_choice()
        n = int(input("Please specify how many recommendations you want as a multiple of 5: "))
        if n % 5 == 0:
            target = list_name[id_num]  
                
            # Loop to get the similarity score for each song against the target
            for i in range(len(list_name)):
                compare = list_name[i]
                if response == 1:
                        metric_inUse = Similarity_metric(list_name, target, compare).euclidean()
                elif response == 2:
                        metric_inUse = Similarity_metric(list_name, target, compare).cosine()
                elif response == 3: # Issues with Pearson, do it manually
                        metric_inUse = Similarity_metric(list_name, target, compare).pearson()
                elif response == 4:
                        metric_inUse = Similarity_metric(list_name, target, compare).jaccard()
                else: # Response is 5
                    metric_inUse = Similarity_metric(list_name, target, compare).manhattan()
                
                results.append(metric_inUse)
    
            # Round each result by 5 decimal places
            results = [i.round(5) for i in results]
            
            if response == 1 or response == 4 or response == 5:
                sorted_results = sorted(range(len(results)), key=lambda x: results[x], reverse=False)
                sorted_results.pop(0) #Drop the first value as we don't need it
            else:
                # Sort the results, if cosine or pearson we need to reverse these results
                sorted_results = sorted(range(len(results)), key=lambda x: results[x], reverse=True)
                sorted_results.pop(0) #Drop the first value as we don't need it
    
            # Print the chosen artist to the user
            print('Your Chosen '+ song_searcher[id_num].getName())
            print('The {} most similar artists to your chosen artist are: '.format(n))

            # Print the n recommendations for the user
            for element in sorted_results:
                print(song_searcher[element].getName())
                j += 1
                if j >= n:
                    break
        else:
            print("Please select an n value as a multiple of 5.")
            Recommendation(list_name).get_artist_recommendation()
    
    def get_song_recommendation(self):
        list_name = self.list_name
        
        # Define needed variables
        results = []
        id_num = int(input("Please enter the ID number of a song: "))
        i = 0
        j = 0
        
        Recommendation(list_name).metric_selection()
        response = Recommendation(list_name).metric_choice()
        n = int(input("Please specify how many recommendations you want as a multiple of 5: "))
        if n % 5 == 0:
            target = list_name[id_num]  
                
            # Loop to get the similarity score for each song against the target
            for i in range(len(list_name)):
                compare = list_name[i]
                if response == 1:
                        metric_inUse = Similarity_metric(list_name, target, compare).euclidean()
                elif response == 2:
                        metric_inUse = Similarity_metric(list_name, target, compare).cosine()
                elif response == 3: # Issues with Pearson, do it manually
                        metric_inUse = Similarity_metric(list_name, target, compare).pearson()
                elif response == 4:
                        metric_inUse = Similarity_metric(list_name, target, compare).jaccard()
                else: # Response is 5
                    metric_inUse = Similarity_metric(list_name, target, compare).manhattan()
            
                results.append(metric_inUse)
    
            # Round each result by 5 decimal places
            results = [i.round(5) for i in results]
            
            if response == 1 or response == 4 or response == 5:
                sorted_results = sorted(range(len(results)), key=lambda x: results[x], reverse=False)
                sorted_results.pop(0) #Drop the first value as we don't need it
            else:
                # Sort the results, if cosine or pearson we need to reverse these results
                sorted_results = sorted(range(len(results)), key=lambda x: results[x], reverse=True)
                sorted_results.pop(0) #Drop the first value as we don't need it
    
            # Print the chosen artist to the user
            print('Your Chosen'+ song_searcher[id_num].getSongName())
            print('The {} most similar songs to your song are: '.format(n))

            # Print the n recommendations for the user
            for element in sorted_results:
                print(song_searcher[element].getSongName(), "by", song_searcher[element].getName())
                j += 1
                if j >= n:
                    break
        else:
            print("Please select an n value as a multiple of 5.")
            Recommendation(list_name).get_song_recommendation()    
    
    def get_target_recommendation(self):
        list_name = self.list_name
        
        # Define needed variables
        results = []
        id_num = int(input("Please enter the ID number of the artist: "))
        i = 0
        j = 0
        
        Recommendation(list_name).metric_selection()
        response = Recommendation(list_name).metric_choice()
        n = int(input("Please specify how many recommendations you want as a multiple of 5: "))
        if n % 5 == 0:
            target = list_name[id_num]  
                
            # Loop to get the similarity score for each song against the target
            for i in range(len(list_name)):
                compare = list_name[i]
                if response == 1:
                        metric_inUse = Similarity_metric(list_name, target, compare).euclidean()
                elif response == 2:
                        metric_inUse = Similarity_metric(list_name, target, compare).cosine()
                elif response == 3: # Issues with Pearson, do it manually
                        metric_inUse = Similarity_metric(list_name, target, compare).pearson()
                elif response == 4:
                        metric_inUse = Similarity_metric(list_name, target, compare).jaccard()
                else: # Response is 5
                    metric_inUse = Similarity_metric(list_name, target, compare).manhattan()
                
                results.append(metric_inUse)
    
            # Round each result by 5 decimal places
            results = [i.round(5) for i in results]
            
            if response == 1 or response == 4 or response == 5:
                sorted_results = sorted(range(len(results)), key=lambda x: results[x], reverse=False)
                sorted_results.pop(0) #Drop the first value as we don't need it
            else:
                # Sort the results, if cosine or pearson we need to reverse these results
                sorted_results = sorted(range(len(results)), key=lambda x: results[x], reverse=True)
                sorted_results.pop(0) #Drop the first value as we don't need it
    
            # Print the chosen artist to the user
            print('Your Chosen '+ song_searcher[id_num].getName())
            print('The {} most similar Tracks to your Artist are: '.format(n))

            # Print the n recommendations for the user
            for element in sorted_results:
                print(song_searcher[element].getName(), song_searcher[element].getSongName())
                j += 1
                if j >= n:
                    break
        else:
            print("Please select an n value as a multiple of 5.")
            Recommendation(list_name).get_target_recommendation()

In [16]:
new = Recommendation(music_df)
new.get_target_recommendation()

Please enter the ID number of the artist: 222
1 Euclidean
2 Cosine
3 Pearson
4 Jaccard
5 Manhattan
Which metric would you like to use from the selection: Enter the number: 7
Your selection is incorrect. Defaulting to Euclidean.
Please specify how many recommendations you want as a multiple of 5: 10
Your Chosen Artist/s: 'Carl Verheyen'
The 10 most similar Tracks to your Artist are: 
Artist/s: 'Elmer Bernstein' Song Name: Clark Street: The Top / Homecoming / Antek's
Artist/s: 'Ramsey Lewis' Song Name: Mother Nature's Son
Artist/s: 'Noe Fajardo Y Su Orquesta' Song Name: Juárez
Artist/s: 'Noe Fajardo Y Su Orquesta' Song Name: España
Artist/s: 'Neal Hefti Orchestra' Song Name: Rhumbacito
Artist/s: 'Lacerenza Michele' Song Name: Concerto per un killer (titoli)
Artist/s: 'Claudio Villa' Song Name: Na sera 'e maggio
Artist/s: 'Benny Goodman Big Band' Song Name: One O'Clock Jump - Live
Artist/s: 'Martin Denny' Song Name: Swamp Fire
Artist/s: 'The Chantays' Song Name: Riders In The Sky


In [17]:
%%time

results = []
id_num = 12310
i = 0

target = music_df[id_num].round(3)

for i in range(len(music_df)):
    compare = music_df[i].round(3)
    results.append(Similarity_metric(music_df, target, compare).jaccard())
    #i += 1 

results = [i.round(5) for i in results]

sorted_results = sorted(range(len(results)), key=lambda x: results[x], reverse=False) #cosine and pearson need reverse true
sorted_results.pop(0) #Drop the first value as we don't need it
sorted_results[0:10]

j = 1
n = 10
final = []

print('Your Chosen Artist/s:', song_searcher[id_num].getName())
print('The {} most similar Tracks to your Artist are: '.format(n))

for element in sorted_results:
    print(song_searcher[element].getName(), song_searcher[element].getSongName(), ', Score:', results[element])
    j += 1
    if j > n:
        break

Your Chosen Artist/s: Artist/s: 'Michael Berresse', 'Bebe Neuwirth'
The 10 most similar Tracks to your Artist are: 
Artist/s: 'Megadeth' Song Name: My Last Words - Remastered , Score: 0.7
Artist/s: 'Linkin Park', 'Zion I' Song Name: Plc.4 Mie Hæd (Amp Live Reanimation) [feat. Zion] , Score: 0.77778
Artist/s: 'Sreeradha Banerjee', 'Saikat Mitra' Song Name: Tomaar Aamaar Mukhetey , Score: 0.77778
Artist/s: 'Ι.Κυριαζής', 'Στ.Χασκίλ', 'Α.Καλδάρας' Song Name: Εβίβα ρεμπέτες , Score: 0.77778
Artist/s: 'Faye Wong' Song Name: 棋子 , Score: 0.77778
Artist/s: 'Caifanes' Song Name: El Elefante , Score: 0.77778
Artist/s: 'Pancho Barraza' Song Name: Formas De Vivir , Score: 0.77778
Artist/s: 'Town Mountain' Song Name: Snowin' On Raton , Score: 0.77778
Artist/s: 'Laura Story' Song Name: Grace , Score: 0.77778
Artist/s: 'Carmen McRae' Song Name: You Took Advantage Of Me , Score: 0.77778
Wall time: 4.01 s


In [ ]:
# %%time 
# # This is to measure how long the calculations take to make improvements if necessary

# # Define variables
# results = []
# id_num = 4391
# i = 0
# j = 0
# n = 10

# # Select the target
# target = music_df[id_num]

# # Loop to get the similarity score for each song against the target
# for i in range(len(music_df)):
#     compare = music_df[i]
#     results.append(Similarity_metric(music_df, target, compare).pearson())
    
# # Round each result by 5 decimal places
# results = [i.round(5) for i in results]

# # Sort the results, if cosine or pearson we need to reverse these results
# sorted_results = sorted(range(len(results)), key=lambda x: results[x], reverse=True)
# sorted_results.pop(0) #Drop the first value as we don't need it
# #print(sorted_results[:10])

# # Print the chosen artist to the user
# print('Your Chosen Artist/s:', song_searcher[id_num].getName())
# print('The {} most similar Tracks to your Artist are: '.format(n))

# # Print the n recommendations for the user
# for element in sorted_results:
#     print('ID:', element, song_searcher[element].getName(), song_searcher[element].getSongName(), ', Score:', results[element])
#     j += 1
#     if j > n:
#         break

## Data Cleaning

In [ ]:
features_df.isnull().sum()
# No empty values present

In [ ]:
features_df.dtypes

In [ ]:
features_df.describe()

# Chosen Method - Content-Based Filtering

In [ ]:
# Testing out 9 feature values vs 9 feature values using the metrics

id_num = 30

one = music_df[id_num].round(3)
two = music_df[90000].round(3)

new_entry = Similarity_metric(music_df, one,two) 
print(new_entry.pearson())
print(new_entry.cosine())
print(new_entry.manhattan())
print(new_entry.euclidean())
print(new_entry.jaccard())
print(one)
print(two)

one = np.sum(one)
two = np.sum(two)

## Part 3 - KNN Recommendation

# Testing section

## Testing Manhattan 

In [ ]:
%%time
# The code basis for one target to all values in the same column

feature = 'Tempo'
feature_list = []
manhattan_list = []
i = 0

for val in features_df[feature]: 
    feature_list.append(Similarity_metric(features_df, [features_df[feature][45000]], [val]))
    manhattan_score = feature_list[i].manhattan()
    manhattan_list.append(manhattan_score)
    i += 1 

In [ ]:
print(len(manhattan_list))
print(manhattan_list[25:30])

In [ ]:
for i in range(0, len(manhattan_list)):
    if abs(manhattan_list[i]) <= 0:
        print(i,':', manhattan_list[i])

## Testing Cosine

In [ ]:
%%time
# The code basis for one target to all values in the same column
 
feature = 'Tempo'
cosine_list = []
i = 0

for val in features_df[feature]:   
    result = (Similarity_metric(features_df, features_df[feature][45000], val)).cosine()
    cosine_list.append(result)
    i += 1


In [ ]:
print(len(cosine_list))
print(cosine_list[25:100])

In [ ]:
addition = 0

for i in range(0, len(cosine_list)):
    if abs(cosine_list[i]) <= 1:
        addition +=1
print(addition)

## Testing Jaccard

In [ ]:
%%time

feature = 'Valence'
feature_list = []
jaccard_list = []
id_num = 45000
i = 0

for val in features_df[feature]:   
    
    feature_list.append(Similarity_metric(features_df, features_df[feature][id_num], val))
    jaccard_score = feature_list[i].jaccard()
    jaccard_list.append(jaccard_score)
    i += 1

In [ ]:
print(len(jaccard_list))
print(jaccard_list[25:35])

In [ ]:
addition = 0

for i in range(0, len(jaccard_list)):
    if jaccard_list[i] <= 0:
        addition +=1
print(addition)

## Testing Pearson

In [ ]:
%%time

id_num = 30

one = music_df[id_num].round(3)
two = music_df[90000].round(3)

new_entry = Similarity_metric(music_df, one,two) 
print(new_entry.pearson())

In [ ]:
print(len(pearson_list))
print(pearson_list[25:35])

In [ ]:
for i in range(0, len(pearson_list)):
    if pearson_list[i] <= 0:
        print(i,':', pearson_list[i])

## Testing Euclidean

In [ ]:
%%time

# The code basis for one target to all values in the same column
 
feature_list = []
euclidean_list = []

feature = 'Tempo'
id_num = 4500 
i = 0 # Index

# lines = iter(open('something.txt', 'r'))
# for val in lines:
#     if val == "!":
#         lines.next()
#         continue
#     <etc>

for val in features_df[feature]: 
    feature_list.append(Similarity_metric(features_df, features_df[feature][id_num], val))
    euclidean_score = feature_list[i].euclidean()
    euclidean_list.append(euclidean_score)
    i += 1 
    
#print(euclidean_list)
#print(tempo_list) #Indent this if you want one-by-one printing
#tempo_list = np.array(tempo_list)

In [ ]:
print(len(euclidean_list))
print(euclidean_list[25:35])

In [ ]:
for i in range(0, len(euclidean_list)):
    if euclidean_list[i] <= 0:
        print(i,':', euclidean_list[i])